In [3]:
from openeye.oechem import *
from openeye.oegraphsim import *

ifs = oemolistream()
ifs.open("/Users/jfeng1/diverse_0.3.sdf")
mol = OEGraphMol()
simMatrix = {}
probes = []
while OEReadMolecule(ifs,mol):
    probes.append(OEGraphMol(mol))
ifs.close()
print len(probes)


32762


In [4]:
ifs = oemolistream()
ifs.open("/Users/jfeng1/mdck_all.sdf")
mol = OEGraphMol()
simMatrix = {}
mols = []
while OEReadMolecule(ifs,mol):
    mols.append(OEGraphMol(mol))
ifs.close()
print len(mols)

fingerprint_db = OEFPDatabase(OEFPType_Circular)

emptyfp = OEFingerPrint()
emptyfp.SetFPTypeBase(fingerprint_db.GetFPTypeBase())

for idx,mol in enumerate(mols):
    fingerprint_db.AddFP(mol)

5804


In [16]:
cutoff = 0.25
def getNeighbors(idx):
    neighbors = []
    mol = probes[idx]
    numFps = fingerprint_db.NumFingerPrints()
    fingerprint_db.SetCutoff(cutoff)
    scores = fingerprint_db.GetScores(mol,0,numFps)
    for score in scores:
        neighbors.append(score.GetIdx())
    return (idx,neighbors)

import time
start = time.time()
import progressbar
import multiprocessing
clusterDict = {}
marker_dict = {}
nProcessor = 5
pool = multiprocessing.Pool(nProcessor)
argList = []
for idx,mol in enumerate(probes):
    marker_dict[idx] = 0
    argList.append(idx)

progressbar = progressbar.ProgressBar(maxval=len(argList))
rs = pool.map_async(getNeighbors,argList,chunksize=1)
progressbar.start()
while True:
    if rs.ready():
        break
    else:
        progress = len(argList)-rs._number_left
        progressbar.update(progress)
        time.sleep(0.5)
result = rs.get()
selectedMols = []
for idx,nbrList in result:
    if len(nbrList) == 0:
        selectedMols.append(probes[idx])

end = time.time()
print 
print end-start

ofs = oemolostream()
ofs.open("/Users/jfeng1/selected_molecules.sdf")
for mol in selectedMols:
    OEWriteMolecule(ofs,mol)
ofs.close()


32.2822320461
